In [1]:
import pandas as pd
import numpy as np
import ssl
import math
import datetime
import urllib.request
import json
from google.colab import files
from datetime import date
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
ssl._create_default_https_context = ssl._create_unverified_context

In [2]:
train = pd.read_csv('https://drive.google.com/uc?export=download&id=1gn6cxKKFWfSfHICPtuCs8YWo3d5SMNqA')
#('https://drive.google.com/uc?export=download&id=1qm5cnqWaqpQChSF9qVHygjUC8W2uwCw7&confirm=t')
train.shape[0]

8573

In [3]:
x = train[['cloudcover', 'dew', 'humidity', 'temp', 'visibility', 'wspd']]
y = train['injured_count']
z = train['dead_count']
w = train['severity']

INJ

In [4]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.1)

In [5]:
m_inj = RandomForestRegressor(n_estimators=90)
m_inj.fit(x_train, y_train)

RandomForestRegressor(n_estimators=90)

In [6]:
m_predict_inj = m_inj.predict(x_test)

In [7]:
go = 0
for i in range(len(m_predict_inj)):
  m_predict_inj[i] = math.trunc(m_predict_inj[i])
  if (y_test.values[i]==math.trunc(m_predict_inj[i])):
    go=go+1;

In [8]:
go/858

0.6794871794871795

In [9]:
mean_squared_error(y_test, m_predict_inj)

0.9988344988344988

In [10]:
pd.DataFrame([y_test.values, m_predict_inj]).T

,0,1
0,0.0,1.0
1,1.0,1.0
2,1.0,1.0
3,1.0,1.0
4,1.0,1.0
...,...,...
853,2.0,1.0
854,2.0,1.0
855,1.0,1.0
856,1.0,1.0


dead

In [11]:
x_train, x_test, z_train, z_test = train_test_split(x, z, test_size = 0.1)

In [12]:
m_dead = RandomForestRegressor(n_estimators=90)
m_dead.fit(x_train, z_train)

RandomForestRegressor(n_estimators=90)

In [13]:
m_predict_dead = m_dead.predict(x_test)

In [14]:
go = 0
for i in range(len(m_predict_dead)):
  m_predict_dead[i] = math.trunc(m_predict_dead[i])
  if (z_test.values[i]==math.trunc(m_predict_dead[i])):
    go=go+1;

In [15]:
go/858

0.9044289044289044

In [16]:
mean_squared_error(z_test, m_predict_dead)

0.1247086247086247

In [17]:
pd.DataFrame([z_test.values, m_predict_dead]).T

,0,1
0,0.0,0.0
1,0.0,0.0
2,0.0,0.0
3,0.0,0.0
4,0.0,0.0
...,...,...
853,0.0,0.0
854,0.0,0.0
855,0.0,0.0
856,0.0,0.0


тяжесть аварии

In [18]:
x_train, x_test, w_train, w_test = train_test_split(x, w, test_size = 0.1)

In [19]:
m_sever = RandomForestRegressor(n_estimators=90)
m_sever.fit(x_train, w_train)

RandomForestRegressor(n_estimators=90)

In [20]:
m_predict_sever = m_sever.predict(x_test)

In [21]:
pd.DataFrame([z_test.values, m_predict_dead]).T

,0,1
0,0.0,0.0
1,0.0,0.0
2,0.0,0.0
3,0.0,0.0
4,0.0,0.0
...,...,...
853,0.0,0.0
854,0.0,0.0
855,0.0,0.0
856,0.0,0.0


Теперь прогнозируем

In [22]:
pre = pd.read_csv('https://drive.google.com/uc?export=download&id=1g6zjANR0GkGOPnAcoSbJJGtOZxDUVvTh')
predict = pd.DataFrame()
print(pre.shape[0])
for i in range(pre.shape[0]):
  date = date.today()
  query_params = '&contentType=json&aggregateMinutes=15&unitGroup=us&includeAstronomy=true&collectStationContributions=true&key={}&startDateTime={}&endDateTime={}&locations={},{}'
  query_params=query_params.format('DE2Q9MGYKB3EEPLKNCG92QSR3', date, date, pre['lat'][i],pre['long'][i])
  response = urllib.request.urlopen('https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/weatherdata/history'+'?'+ query_params)
  data = response.read()
  weatherData = json.loads(data.decode('utf-8'))
  try:
    weather = weatherData['locations'][f"{pre['lat'][i]},{pre['long'][i]}"]
    print(weather['values'][0]['visibility'],"->",i)
    df_small = {'lat':pre['lat'][i], 'long':pre['long'][i], 'cloudcover': 82, 'dew': weather['values'][0]['dew'], 'humidity': weather['values'][0]['humidity'], 'temp': weather['values'][0]['temp'], 'visibility': weather['values'][0]['visibility'], 'wspd': weather['values'][0]['wspd'] }
    predict=predict.append(df_small, ignore_index=True)
    print("все окей","->",i)
  except KeyError:
    print("нет данных","->",i)
#predict.drop(columns = ['Unnamed: 0'],axis = 1, inplace=True)

34
6.2 -> 0
все окей -> 0
6.2 -> 1
все окей -> 1
6.2 -> 2
все окей -> 2
6.2 -> 3
все окей -> 3
6.2 -> 4
все окей -> 4
6.2 -> 5
все окей -> 5
6.2 -> 6
все окей -> 6
6.2 -> 7
все окей -> 7
6.2 -> 8
все окей -> 8
6.2 -> 9
все окей -> 9
6.2 -> 10
все окей -> 10
6.2 -> 11
все окей -> 11
6.2 -> 12
все окей -> 12
6.2 -> 13
все окей -> 13
6.2 -> 14
все окей -> 14
6.2 -> 15
все окей -> 15
6.2 -> 16
все окей -> 16
6.2 -> 17
все окей -> 17
6.2 -> 18
все окей -> 18
6.2 -> 19
все окей -> 19
6.2 -> 20
все окей -> 20
6.2 -> 21
все окей -> 21
6.2 -> 22
все окей -> 22
6.2 -> 23
все окей -> 23
6.2 -> 24
все окей -> 24
6.2 -> 25
все окей -> 25
6.2 -> 26
все окей -> 26
6.2 -> 27
все окей -> 27
6.2 -> 28
все окей -> 28
6.2 -> 29
все окей -> 29
6.2 -> 30
все окей -> 30
6.2 -> 31
все окей -> 31
6.2 -> 32
все окей -> 32
6.2 -> 33
все окей -> 33


In [23]:
x = predict[['cloudcover', 'dew', 'humidity', 'temp', 'visibility', 'wspd']]
m_predict_inj = m_inj.predict(x)
m_predict_dead = m_dead.predict(x)
m_predict_sever = m_sever.predict(x)

In [24]:
for i in range(len(m_predict_inj)):
  m_predict_inj[i] = math.trunc(m_predict_inj[i])
  m_predict_dead[i] = math.trunc(m_predict_dead[i])
  m_predict_sever[i] = math.trunc(m_predict_sever[i])

In [25]:
#Легкий-1
#С погибшими-2
#Тяжёлый-3
final = pd.DataFrame()
for i in range(len(pre)):
  df_small = {'lat':pre['lat'][i], 'long':pre['long'][i], 'cloudcover': 82, 'dew': weather['values'][0]['dew'], 'humidity': weather['values'][0]['humidity'], 'temp': weather['values'][0]['temp'], 'visibility': weather['values'][0]['visibility'], 'wspd': weather['values'][0]['wspd'], 'inj':m_predict_inj[i], 'dead':m_predict_dead[i], 'severity':m_predict_sever[i]}
  final = final.append(df_small, ignore_index=True)
final

,lat,long,cloudcover,dew,humidity,temp,visibility,wspd,inj,dead,severity
0,55.765503,37.789890,82.0,49.0,59.83,63.2,6.2,13.8,1.0,0.0,1.0
1,55.781565,37.862331,82.0,49.0,59.83,63.2,6.2,13.8,1.0,0.0,1.0
2,55.793753,37.931338,82.0,49.0,59.83,63.2,6.2,13.8,1.0,0.0,1.0
3,55.800135,38.000346,82.0,49.0,59.83,63.2,6.2,13.8,1.0,0.0,1.0
4,55.809803,38.082401,82.0,49.0,59.83,63.2,6.2,13.8,1.0,0.0,1.0
5,55.817729,38.152438,82.0,49.0,59.83,63.2,6.2,13.8,1.0,0.0,1.0
6,55.823528,38.203250,82.0,49.0,59.83,63.2,6.2,13.8,1.0,0.0,2.0
7,55.827393,38.253719,82.0,49.0,59.83,63.2,6.2,13.8,1.0,0.0,1.0
8,55.830292,38.302470,82.0,49.0,59.83,63.2,6.2,13.8,1.0,0.0,1.0
9,55.833576,38.353282,82.0,49.0,59.83,63.2,6.2,13.8,1.0,0.0,1.0


In [26]:
final.to_json(r'accidents.json',orient="records")
files.download('accidents.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>